# Multidimensional Least-Squares Kronecker Factorization (MLS-KronF)

In [2]:
import numpy as np
import tensorly as tl
from tensorly.tenalg import mode_dot
from numpy.random import randn, rand, randint
from numpy.linalg import svd, matrix_rank, norm
from scipy.io import loadmat
import time

## Problem 1

In [3]:
def error(X_0, X_0_hat):
    return norm(X_0_hat - X_0, 'fro')**2/norm(X_0, 'fro')**2

def Hermitian(X):
    return np.array(np.matrix(X).H)

def HOSVD(X, ml_rank=None):
    U_list = []
    S = X
    
    for n in range(X.ndim):
        X_n = tl.unfold(X, n)
        Un, _, _ = svd(X_n)
        
        if ml_rank:
            Un = Un[:, :ml_rank[n]]

        U_list.append(Un)

        S = mode_dot(S, Hermitian(Un), n)
        
    return S, U_list

In [4]:
# P = [2, 2, 2]
# Q = [2, 2, 2]
# A = randint(1, 4, size=(P[0], P[0]))
# B = randint(1, 4, size=(P[1], P[1]))
# C = randint(1, 4, size=(P[2], P[2]))

# # M = np.kron(np.kron(A, B), C)
# M = np.reshape([i+1 for i in range(64)], (8, 8))


In [5]:
P = [2, 3, 4]
Q = [2, 3, 4]
A = randn(P[0], Q[0]) + randn(P[0], Q[0])*1j
B = randn(P[1], Q[1]) + randn(P[1], Q[1])*1j
C = randn(P[2], Q[2]) + randn(P[2], Q[2])*1j

M = np.kron(np.kron(A, B), C)


In [6]:
def MLS_KronF(M, P, Q):
    X_reshaped = []
    X_dash = []
    for M_v in np.hsplit(M, P[0]):
        for M_h in np.vsplit(M_v, Q[0]):       

            X_dash_flat = []
            for M1_v in np.hsplit(M_h, P[1]):
                for M1_h in np.vsplit(M1_v, Q[1]):
                    X_dash_flat.append(M1_h.T.flatten())

            X_dash.append(np.array(X_dash_flat))

    for X_ in X_dash:
        X_reshaped.append(X_.flatten())

    X_reshaped = np.array(X_reshaped).T

    S, U = HOSVD(tl.fold(X_reshaped, 0, (P[1]*Q[1], P[2]*Q[2], P[0]*Q[0])))

    As = ((S.flat[0]**(1/3))*U[2][:, 0]).reshape(P[0], Q[0]).T

    Bs = ((S.flat[0]**(1/3))*U[0][:, 0]).reshape(P[1], Q[1]).T

    Cs = ((S.flat[0]**(1/3))*U[1][:, 0]).reshape(P[2], Q[2]).T
    
    return As, Bs, Cs

In [7]:
As, Bs, Cs = MLS_KronF(M, P, Q)

norm(M - np.kron(np.kron(As, Bs), Cs), 'fro')

4.4769128810892264e-14

In [8]:
A

array([[ 1.70026052-1.47999832j, -0.62899487-0.54327142j],
       [ 0.34936774+1.1005918j , -0.5874927 -0.58082163j]])

In [9]:
As

array([[-2.47871595+2.01699549j,  0.86565486+0.79935013j],
       [-0.44323185-1.57585479j,  0.80508411+0.85060192j]])

## Problem 2

In [724]:
P = [2, 3, 4]
Q = [2, 3, 4]

SNRdb_range = [0, 5, 10, 15, 20, 25, 30]
No_experiments = 1000

NMSE = []

for SNRdb in SNRdb_range:
    exp_errors = [] # list of errors for the experiments
    
    for i in range(No_experiments):
        A = randn(P[0], Q[0]) + randn(P[0], Q[0])*1j
        B = randn(P[1], Q[1]) + randn(P[1], Q[1])*1j
        C = randn(P[2], Q[2]) + randn(P[2], Q[2])*1j

        X_0 = np.kron(np.kron(A, B), C)
        
        V = randn(P[0]*P[1]*P[2], Q[0]*Q[1]*Q[2]) + randn(P[0]*P[1]*P[2], Q[0]*Q[1]*Q[2])*1j
        
        # calculate alpha and X
        alpha = np.sqrt((1/10**(SNRdb/10))*(norm(X_0, 'fro')**2/norm(V, 'fro')**2))
        X = X_0 + alpha*V
        
        # Estimante via MLS-KRF
        As, Bs, Cs = MLS_KronF(X, P, Q)
        
        X_0_hat = np.kron(np.kron(As, Bs), Cs)
        
        exp_errors.append(error(X_0, X_0_hat))
        
    NMSE.append(np.sum(exp_errors)/No_experiments)

In [725]:
NMSE

[0.05146159337975077,
 0.015235815690828582,
 0.004730053896320703,
 0.0014907710117227305,
 0.00046612602300219306,
 0.0001486261566570544,
 4.6874526245452046e-05]

In [726]:
import plotly.graph_objects as go

fig = go.Figure(
    layout=go.Layout(
        title="NMSE vs SNR curve",
        template="simple_white"))

fig.add_trace(go.Scatter(x=SNRdb_range, y=NMSE, mode='lines+markers', name='MLS-KronF'))

fig.update_yaxes(title='NMSE',showgrid=True,type="log")
fig.update_xaxes(title='SNR(dB)')

fig.show()